In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

/home/mathieu/Prose/Mathieu/Benter-Project


In [2]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from winning_validation import errors
from winning_validation import r_squared
from winning_horse_models import sklearn
from winning_horse_models.dl_shared_layers import LogisticRegressionModel, DLSharedLayersModel, DLLayersGeneratorModel
from winning_horse_models.xgboost import XGBoostWinningModel
from winning_horse_models.catboost import CatboostWinningModel
from winning_horse_models.lgbm import LGBMWinningModel
from training_procedures import sequential_training, flattened_training
from constants import Sources, SplitSets, XFormats, YFormats
from catboost import CatBoostClassifier
from utils import preprocess
from sklearn.feature_selection import SequentialFeatureSelector

from database.setup import create_sqlalchemy_session
from models.race import Race
from models.runner import Runner
from sklearn.linear_model import SGDClassifier
tqdm.pandas()

import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

/home/mathieu/.pyenv/versions/3.7.9/envs/benter-project_venv/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
SOURCE = Sources.UNIBET
N_FEATURES = preprocess.get_n_preprocessed_feature_columns(source=SOURCE)

# DLSharedLGBMWinningModel

In [ ]:
%%time
%%capture --no-stdout
space = {'layers':[{'type':'Dense', 'n_units':1}]}

model =DLLayersGeneratorModel(source=SOURCE, n_features = N_FEATURES, hyperparameters=space, name='1')
    
model, _ = sequential_training.train_on_n_horses(source=SOURCE, winning_model=model, n_horses=10, n_epochs=0, start_training_at=dt.datetime.now(),
                                                 n_epochs_per_n_horses=1, verbose=True)

res=r_squared.compute_mcfadden_r_squared_on_n_horses(source=SOURCE,winning_model=model, n_horses=10, verbose=True)
print(res)

In [4]:
x, y, _ = import_data.get_races_per_horse_number(
    source=SOURCE,
    n_horses=10,
    on_split=SplitSets.TRAIN,
    x_format=XFormats.SEQUENTIAL,
    y_format=YFormats.INDEX_FIRST,
    preprocessing=True,
)

In [5]:
x.shape

(13625, 10, 120)

In [8]:
from sklearn.decomposition import FastICA

In [10]:
dim_reduction_model=FastICA(n_components=10)

In [14]:
horse_x=x.reshape(x.shape[0]*x.shape[1], x.shape[2])

In [15]:
dim_reduction_model.fit(horse_x)

FastICA(n_components=10)

In [16]:
dim_reduction_model

FastICA(n_components=10)

In [ ]:
class DLLayersGeneratorModelDminReduction(SequentialMixin, AbstractWinningModel):

    _NotFittedModelError = _ShouldNotBeTriggeredException

    def __init__(
        self,
        name: str,
        hyperparameters: Optional[dict] = None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.name = name
        self._layers_per_n_horses: Dict[int, Any] = {}
        self._dim
        self._hyperparameters = hyperparameters

    def get_layers(self, hyperparameters: Optional[dict] = None):
        if hyperparameters is None:
            return tf.keras.layers.Dense(1)
        if len(hyperparameters["layers"]) == 1:
            layer = hyperparameters["layers"][0]
            assert layer["type"] == "Dense"
            return tf.keras.layers.Dense(layer["n_units"])

        assert hyperparameters["layers"][-1]["n_units"] == 1
        layers = []
        for layer in hyperparameters["layers"]:
            assert layer["type"] in ("Dense", "Dropout")
            if layer["type"] == "Dense":
                params = {"units": layer["n_units"]}
                if "kernel_regularizer" in layer:
                    assert layer["kernel_regularizer"]["type"] == "l2"
                    params["kernel_regularizer"] = tf.keras.regularizers.l2(
                        l2=layer["kernel_regularizer"]["l2"]
                    )
                layers.append(tf.keras.layers.Dense(**params))
                continue
            layers.append(tf.keras.layers.Dropout(layer["rate"]))
        return tf.keras.Sequential(layers=layers)

    def _create_n_horses_model(self, n_horses: int):
        self._layers_per_n_horses[n_horses] = self.get_layers(self._hyperparameters)
        inputs = tf.keras.Input(shape=(n_horses, self.n_features))
        unstacked = tf.keras.layers.Lambda(lambda x: tf.unstack(x, axis=1))(inputs)
        dense_outputs = [
            self._layers_per_n_horses[n_horses](x) for x in unstacked
        ]  # our generated layer
        merged = tf.keras.layers.Lambda(lambda x: tf.stack(x, axis=1))(dense_outputs)
        outputs = tf.keras.layers.Reshape(target_shape=(n_horses,))(merged)
        outputs = tf.keras.layers.Lambda(
            lambda x: tf.keras.activations.softmax(x, axis=-1)
        )(outputs)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        model.compile(
            loss="categorical_crossentropy",
            optimizer="rmsprop",
            metrics=["categorical_accuracy", "categorical_crossentropy"],
        )
        model.build(input_shape=(None, n_horses, self.n_features))
        return model

    def save_model(self, prefix: Optional[str] = None):
        raise NotImplementedError

    @classmethod
    def load_model(cls, name: str, prefix: Optional[str] = None):
        raise NotImplementedError